### Data Engineering

In [14]:
import pandas as pd 

from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

In [9]:
train = pd.read_csv('././dataset/train.csv')
test = pd.read_csv('././dataset//test.csv')

In [10]:
class CFG:
    debug=False
    apex=True
    print_freq=100
    num_workers=4
    #model="AI-Growth-Lab/PatentSBERTa_Kaggle_V3"
    #model="google/bigbird-pegasus-large-bigpatent"
    #model="google/pegasus-big_patent"
    #model="Yanhao/simcse-bert-for-patent"
    model="prithivida/bert-for-patents-64d"
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0.1
    epochs=10
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=8
    fc_dropout=0.2
    target_size=1
    max_len=1024
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    n_fold=4
    trn_fold=[3]
    train=True
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [11]:
cpc_texts = torch.load("././dataset/cpc_texts.pth")
train['context_text'] = train['context'].map(cpc_texts).str.lower()
test['context_text'] = test['context'].map(cpc_texts).str.lower()

In [12]:
train['text'] = train['anchor'] + '[SEP]' + train['target'] + '[SEP]'  + train['context_text']
test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
display(train.head())
display(test.head())

,id,anchor,target,context,score,context_text,text
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,human necessities. furniture; domestic article...,abatement[SEP]abatement of pollution[SEP]human...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,human necessities. furniture; domestic article...,abatement[SEP]act of abating[SEP]human necessi...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,human necessities. furniture; domestic article...,abatement[SEP]active catalyst[SEP]human necess...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,human necessities. furniture; domestic article...,abatement[SEP]eliminating process[SEP]human ne...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,human necessities. furniture; domestic article...,abatement[SEP]forest region[SEP]human necessit...


,id,anchor,target,context,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,physics. optics,opc drum[SEP]inorganic photoconductor drum[SEP...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,mechanical engineering; lighting; heating; wea...,adjust gas flow[SEP]altering gas flow[SEP]mech...
2,36baf228038e314b,lower trunnion,lower locating,B60,performing operations; transporting. vehicles ...,lower trunnion[SEP]lower locating[SEP]performi...
3,1f37ead645e7f0c8,cap component,upper portion,D06,textiles; paper. treatment of textiles or the ...,cap component[SEP]upper portion[SEP]textiles; ...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,electricity. electric communication technique,neural stimulation[SEP]artificial neural netwo...


In [15]:
train['score_map'] = train['score'].map({0.00: 0, 0.25: 1, 0.50: 2, 0.75: 3, 1.00: 4})
Fold = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(train, train['score_map'])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())

fold
0    9119
1    9118
2    9118
3    9118
dtype: int64